# ORCID_migrations_2018 add more fields

base on John Bohannon's script (https://datadryad.org/stash/downloads/file_stream/65430)

Since 2018, ORCID public data started to use XSD v2_0, 
And the archived file no longer contains JSON.
In order to reuse John's script, we have to convert ORCID's XML file to JSON and modify the extraction function.


ORCID Conversion Library(https://github.com/ORCID/orcid-conversion-lib)

XSD v2_0(https://github.com/ORCID/ORCID-Source/tree/master/orcid-model/src/main/resources/record_2.0)

In [81]:
import json, os, sys
import pandas as pd

def file_generator_from_summary(summary_dir):
    flag=-1
    ''' Using a generator allows pausing and restarting
    without having to figure out where you left off. '''
    for z, json_dir in enumerate(os.listdir(summary_dir)):
        if os.path.isdir(os.path.join(summary_dir, json_dir)):
            for n, i in enumerate(os.listdir(os.path.join(summary_dir, json_dir))):
                if os.path.isfile(os.path.join(summary_dir,json_dir, i)):
                    flag=flag+1
                    yield (flag, os.path.join(summary_dir,json_dir, i))
            
def file_generator(json_dir):
    ''' Using a generator allows pausing and restarting
    without having to figure out where you left off. '''
    for n, i in enumerate(os.listdir(json_dir)):
        yield (n, os.path.join(json_dir, i))

def get_profiles(dataA, dataP, dataW, json_files, stop = None):
    ''' Iterate over JSON files and process them '''
    for n, filepath in json_files:
        # keep track of progress
        if n%5000==0:
            sys.stdout.flush()
            sys.stdout.write('\r{}'.format(n))
        # terminate if stop is specified and reached
        if stop is not None and n >= stop:
            return
        # process this JSON file and harvest the data
        if filepath.endswith(".json"):
            with open(filepath) as f:
                profile = json.load(f)
                for row in get_affiliations(profile):
                    dataA.append(row)
                w = get_works(profile)
                if w is not None:
                    for row in w:
                        dataW.append(row)
                p = get_person(profile)
                if p is not None:
                    if p[1] or p[2]:
                        dataP.append(p)
                    
def has_education(profile):
    ''' This tests whether the profile is valid and has education or employe info '''
    try:
        if profile.get('error-code') is None and profile['activities-summary']['educations']['education-summary'] :
            return True
    except:
        return False
    
def has_employment(profile):
    ''' This tests whether the profile is valid and has education or employe info '''
    try:
        if profile.get('error-code') is None and profile['activities-summary']['employments']['employment-summary'] :
            return True
    except:
        return False
    
def has_person(profile):
    ''' This tests whether the profile is valid and has education or employe info '''
    try:
        if profile.get('error-code') is None and profile['addresses']['address'] :
            return True
    except:
        return False
    
def get_date(row):
    dateStr=""
    try:
        if row["year"]["value"] is not None:
            dateStr+=row["year"]["value"]
    except:
        return None
    try:
        if row["month"]["value"] is not None:
            dateStr+=row["month"]["value"]
    except:
        dateStr+="MM"
    try:
        if row["day"]["value"] is not None:
            dateStr+=row["day"]["value"]
    except:
        dateStr+="dd"
    return dateStr
            
def get_person(profile):
    try:
        if profile.get('error-code') is None and profile["orcid-identifier"]["path"] :
            orcid_id = profile["orcid-identifier"]["path"]
            row = [orcid_id]
            try:
                row.append(profile["person"]["biography"]["content"])
            except:
                row.append(None)
            try:
                keyword=""
                for k in profile['person']['keywords']['keyword']:
                    keyword+="|"+k['content']
                row.append(keyword)
            except:
                row.append(None)
            return row
    except:
        return None

def get_works(profile):
    try:
        if profile.get('error-code') is None and profile["activities-summary"]["works"]["group"] :
            profile_data = []
            orcid_id = profile["orcid-identifier"]["path"]
            for workgroup in profile["activities-summary"]["works"]["group"]:
                for w in workgroup['work-summary']:
                    row = [orcid_id]
                    try:
                        row.append(get_date(w['publication-date']))
                    except:
                        row.append(None)
                    try:
                        row.append(w['type'])
                    except:
                        row.append(None)
                    try:
                        row.append(w['put-code'])
                    except:
                        row.append(None)
                    profile_data.append(row)
            return profile_data
    except:
        return None
    
def get_affiliations(profile):
    ''' For each profile, extract all affiliations and metadata '''
    profile_data = []
    #in 2018+ affiliations info separated to two keys as educations and employments
    if has_education(profile):
        orcid_id = profile["orcid-identifier"]["path"]
        try:
            for aff in profile['activities-summary']['educations']['education-summary']:
                row = [orcid_id]
                row.append(aff["organization"]["address"]["country"])
                try:
                    row.append(aff["organization"]["name"])
                except:
                    row.append(None)
                try:
                    row.append(aff["organization"]["disambiguated-organization"]["disambiguated-organization-identifier"])
                except:
                    row.append(None)
                try:
                    row.append(aff["start-date"]["year"]["value"])
                except:
                    row.append(None)
                try:
                    row.append(aff["end-date"]["year"]["value"])
                except:
                    row.append(None)
                try:
                    row.append("EDUCATION")
                except:
                    row.append(None)
                try:
                    row.append(aff["role-title"])
                except:
                    row.append(None)
                ##add gmt_modified
                try:
                    row.append(aff["last-modified-date"]["value"])
                except:
                    row.append(None)
                ##add department-name
                try:
                    row.append(aff["department-name"])
                except:
                    row.append(None)
                ##add visibility
                try:
                    row.append(aff["visibility"])
                except:
                    row.append(None)
                ##add city and region
                try:
                    row.append(aff["organization"]["address"]["city"])
                except:
                    row.append(None)
                try:
                    row.append(aff["organization"]["address"]["region"])
                except:
                    row.append(None)
                ##add start-date and end-date yyyyMMdd
                try:
                    row.append(get_date(aff["start-date"]))
                    #row.append(None)
                except:
                    row.append(None)
                try:
                    row.append(get_date(aff["end-date"]))
                    #row.append(None)
                except:
                    row.append(None)
                profile_data.append(row)
        except:
            pass
    if has_employment(profile):
        orcid_id = profile["orcid-identifier"]["path"]
        try:
            for aff in profile['activities-summary']['employments']['employment-summary']:
                row = [orcid_id]
                row.append(aff["organization"]["address"]["country"])
                try:
                    row.append(aff["organization"]["name"])
                except:
                    row.append(None)
                try:
                    row.append(aff["organization"]["disambiguated-organization"]["disambiguated-organization-identifier"])
                except:
                    row.append(None)
                try:
                    row.append(aff["start-date"]["year"]["value"])
                except:
                    row.append(None)
                try:
                    row.append(aff["end-date"]["year"]["value"])
                except:
                    row.append(None)
                try:
                    row.append("EMPLOYMENT")
                except:
                    row.append(None)
                try:
                    row.append(aff["role-title"])
                except:
                    row.append(None)
                ##add gmt_modified
                try:
                    row.append(aff["last-modified-date"]["value"])
                except:
                    row.append(None)
                ##add department-name
                try:
                    row.append(aff["department-name"])
                except:
                    row.append(None)
                ##add visibility
                try:
                    row.append(aff["visibility"])
                except:
                    row.append(None)
                ##add city and region
                try:
                    row.append(aff["organization"]["address"]["city"])
                except:
                    row.append(None)
                try:
                    row.append(aff["organization"]["address"]["region"])
                except:
                    row.append(None)
                ##add start-date and end-date yyyyMMdd
                try:
                    row.append(get_date(aff["start-date"]))
                    #row.append(None)
                except:
                    row.append(None)
                try:
                    row.append(get_date(aff["end-date"]))
                    #row.append(None)
                except:
                    row.append(None)
                profile_data.append(row)
        except:
            pass
    if has_person(profile):
        orcid_id = profile["orcid-identifier"]["path"]
        try:
            for aff in profile['person']['addresses']['address']:
                row = [orcid_id]
                row.append(aff["country"]["value"])
                try:
                    row.append(aff["source"]["source-name"]["value"])
                except:
                    row.append(None)
                try:
                    #row.append(aff["organization"]["disambiguated-organization"]["disambiguated-organization-identifier"])
                    row.append(None)
                except:
                    row.append(None)
                try:
                    #row.append(aff["start-date"]["year"]["value"])
                    row.append(None)
                except:
                    row.append(None)
                try:
                    #row.append(aff["end-date"]["year"]["value"])
                    row.append(None)
                except:
                    row.append(None)
                try:
                    row.append("PERSON")
                except:
                    row.append(None)
                try:
                    #row.append(aff["role-title"])
                    row.append(None)
                except:
                    row.append(None)
                ##add gmt_modified
                try:
                    row.append(aff["last-modified-date"]["value"])
                except:
                    row.append(None)
                ##add department-name
                try:
                    #row.append(aff["department-name"])
                    row.append(None)
                except:
                    row.append(None)
                ##add visibility
                try:
                    row.append(aff["visibility"])
                except:
                    row.append(None)
                ##add city and region
                try:
                    row.append(aff["city"]["value"])
                except:
                    row.append(None)
                try:
                    row.append(aff["region"]["value"])
                except:
                    row.append(None)
                ##add start-date and end-date yyyyMMdd
                try:
                    #row.append(aff["start-date"]["year"]["value"])
                    row.append(None)
                except:
                    row.append(None)
                try:
                    #row.append(aff["end-date"]["year"]["value"])
                    row.append(None)
                except:
                    row.append(None)
                profile_data.append(row)
        except:
            pass
    return profile_data

In [88]:
#json_dir ="/Users/qianshaoli/ORCID_2018/000" 
#json_files = file_generator(json_dir)
#summary_dir ="/Users/qianshaoli/ORCID_2018_test/" 
summary_dir ="/Users/qianshaoli/ORCID_2018/" 
json_files = file_generator_from_summary(summary_dir)
dataA = []
dataP = []
dataW = []

In [89]:
%%time
get_profiles(dataA, dataP, dataW, json_files)

5380000CPU times: user 1h 31min 5s, sys: 38min 28s, total: 2h 9min 34s
Wall time: 3h 3min 16s


In [90]:
df=[]
df = pd.DataFrame(dataA, columns = ["orcid_id", "country", "organization_name", 
                              "Ringgold_id", "start_year", "end_year", 
                              "affiliation_type", "affiliation_role","gmt_modified","department","visibility","city","region","start_date","end_date"])
df.head()

,orcid_id,country,organization_name,Ringgold_id,start_year,end_year,affiliation_type,affiliation_role,gmt_modified,department,visibility,city,region,start_date,end_date
0,0000-0002-9418-3797,EG,Ain Shams University,68791,None,None,EDUCATION,PhD,1448730441029,Horticulture,PUBLIC,Cairo,None,None,None
1,0000-0002-9418-3797,EG,National Research Centre,68787,None,None,EMPLOYMENT,None,1448730520502,Plant Biotechnology,PUBLIC,Giza,None,None,None
2,0000-0001-6920-0797,BR,Universidade Federal de Pernambuco - UFPE,None,2017,None,EDUCATION,Mestrando em Geografia - PPGEO,1520799749026,Ciências Geográficas - DCG/CFCH,PUBLIC,Recife,PE/NE,20170305,None
3,0000-0001-6920-0797,BR,Universidade Federal de Pernambuco - UFPE,None,2013,2017,EDUCATION,Licenciatura em Geografia,1520799751250,Ciências Geográficas - DCG/CFCH,PUBLIC,Recife,PE/NE,20130520,20170217
4,0000-0001-6920-0797,BR,Universidade Federal de Pernambuco - UFPE,133639,2017,None,EMPLOYMENT,REPRESENTANTE DISCENTE DO MESTRADO TURMA 2017-...,1520800374502,Departamento de Ciências Geográficas - DCG,PUBLIC,Recife,PE/NE,20170527,None


In [92]:
len(df)

4590010

In [93]:
dfP=[]
dfP = pd.DataFrame(dataP, columns = ["orcid_id", "biography", "keywords"])
dfP.head(10)

,orcid_id,biography,keywords
0,0000-0002-2917-0797,None,"|finance, computer science, derivatives"
1,0000-0002-7543-3797,Prof. D. A. Sun received his PhD in geotechnic...,"|Unsaturated soil mechanics, Constitutive mode..."
2,0000-0002-8398-6797,None,"|cosmology, astronomy, astrophysics, dark ener..."
3,0000-0001-7223-1797,None,|Stimuli Drug delivery|Upvonversioin nanoparti...
4,0000-0002-6335-2797,None,"|Analytical Chemistry, Electroanalytcial Chemi..."
5,0000-0002-6846-0797,"szkolenia na tyczkach, wyjazdy narciarskie",None
6,0000-0002-3374-3797,"Diploma in physics, RWTH Aachen, Germany (1984...","|statistical physics, physical chemistry, mole..."
7,0000-0001-6855-8797,Work Experience\nApril 2008 - July 2008: Resea...,"|Water resources management, Participatory Irr..."
8,0000-0002-7133-6797,None,"|Environment, Environmental Management Control..."
9,0000-0001-5194-0797,Date of birth : 01.03.1960.\nHigher education ...,None


In [94]:
dfP[dfP.orcid_id == "0000-0003-2043-0000"]

,orcid_id,biography,keywords
133549,0000-0003-2043-0000,None,|Periodontology|Immunology|Dementia|Porphyromo...


In [95]:
dfW=[]
dfW = pd.DataFrame(dataW, columns = ["orcid_id", "publication_date","type", "put_code"])
dfW.head(10)

,orcid_id,publication_date,type,put_code
0,0000-0002-5636-3797,2018MMdd,JOURNAL_ARTICLE,38913823
1,0000-0002-5636-3797,2013MMdd,JOURNAL_ARTICLE,38913822
2,0000-0002-7543-3797,20180511,JOURNAL_ARTICLE,35912525
3,0000-0002-7543-3797,201711dd,JOURNAL_ARTICLE,32161338
4,0000-0002-7543-3797,2017MMdd,JOURNAL_ARTICLE,31542445
5,0000-0002-7543-3797,2016MMdd,JOURNAL_ARTICLE,31542451
6,0000-0002-7543-3797,2016MMdd,JOURNAL_ARTICLE,22045302
7,0000-0002-7543-3797,2016MMdd,JOURNAL_ARTICLE,31542459
8,0000-0002-7543-3797,2016MMdd,JOURNAL_ARTICLE,31542462
9,0000-0002-7543-3797,2016MMdd,CONFERENCE_PAPER,31542441


In [96]:
dfW[dfW.orcid_id=="0000-0003-2146-8000"].put_code.nunique()

133

In [97]:
df.orcid_id.nunique(), len(df)

(1627845, 4590010)

There are 1627845 researchers with ORCID profiles that include affiliations data, with a total of 4590010 affiliations over the course of their careers.

In [98]:
education_without_dates = df[((df.affiliation_type == "EDUCATION") & 
                       (df.start_year.isnull()) & 
                       (df.end_year.isnull()))]
len(education_without_dates), education_without_dates.orcid_id.nunique()

(159866, 135293)

There are 135293 profiles that include EDUCATION affiliations that do not have any start_date or end_date. These researchers are just listing their education affiliations without noting the year they obtained the degree.

These data stop in December 2018. So affiliations that do have a start_year but no end_year are ongoing affilations. How many are there?

In [99]:
ongoing = df[((df.start_year.notnull()) & (df.end_year.isnull()))]
len(ongoing), ongoing.orcid_id.nunique()

(1440965, 1175275)

There are 1175275 people with ongoing affiliations as of 2018. Let's recode those missing end_year values of ongoing affiliations as 2018.

In [100]:
def recode_ongoing(row):
    ''' Recode the end_year value of some affiliations to 2018 
        since they are ongoing affiliations. '''
    if row.start_year is not None and row.end_year is None:
        return 2018
    else:
        return row.end_year
    
df.end_year = df.apply(recode_ongoing, axis = 1)

Next we should exclude people whose country of residence in 2018 is ambiguous, i.e. those with more than one country affilation. How many are there?

In [176]:
today_affs = df[df.end_year == 2018]
g = today_affs.groupby(["orcid_id", "country"]).aggregate(len).reset_index()
multi_2018_country_people = set(g[g.duplicated("orcid_id")].orcid_id)

len(multi_2018_country_people)

29075

There are 29075 of these ambiguous people. Let's exclude them, since for the analysis of migrations, we don't know where these people ended up in the present day.

In [178]:
##df = df[~df.orcid_id.isin(multi_2018_country_people)]

Next, we want to know which of these people got PhD degrees. So let's classify the affiliation_role of the EDUCATION affiliations as being PhD degrees or not, just using simple string matching. I built this classifier through trial and error on these data. I have not yet found an exception, but there are probably some in there. The error rate should be very low.

In [101]:
def is_phd(role):
    ''' After lowercasing the affiliation_role string, look for
        terms that indicate it is a Ph.D. degree.
        Yes, regular expressions are tidier, but also unreadable 
        to most people and harder to debug. '''
    # These are international synonyms for the Ph.D. degree
    synonyms = ("phd", "ph.d","ph. d", "dphil", "d.phil", "rer. nat", 
                "rer, nat", "doctor rerum", "doktor rerum",u'博士')
    # This catches things like "Doctorate en Chimie" but
    # excludes "Postdoctoral Fellow" and "Medical Doctorate"
    special_cases_allowed = ("doctor", "doktor")
    special_cases_disallowed = ("pre", "post", "med",u'在读')
    if (type(role) == str or type(role) == unicode):
        # lowercase the string
        role = role.lower()
        # Look for Ph.D. synonyms
        if any([(i in role) for i in synonyms]):
            return True
        # Look for special cases
        if any([(i in role) for i in special_cases_allowed]) and \
        not any([(i in role) for i in special_cases_disallowed]):
            return True
    # Otherwise call it False
    return False

df["is_phd"] = df.affiliation_role.apply(is_phd)

def is_master(role):
    # These are international synonyms for the master degree
    synonyms = ("master", "ms","ma", "msc",u'硕士',"m.a","m.s")
    # This catches things like "Doctorate en Chimie" but
    # excludes "Postdoctoral Fellow" and "Medical Doctorate"
    special_cases_allowed = ("mba", "mfa")
    special_cases_disallowed = ("pre", "post", "med",u'在读')
    if (type(role) == str or type(role) == unicode):
        # lowercase the string
        role = role.lower()
        # Look for master synonyms
        if any([(i in role) for i in synonyms]):
            return True
        # Look for special cases
        if any([(i in role) for i in special_cases_allowed]) and \
        not any([(i in role) for i in special_cases_disallowed]):
            return True
    # Otherwise call it False
    return False

df["is_master"] = df.affiliation_role.apply(is_master)
df.head()

,orcid_id,country,organization_name,Ringgold_id,start_year,end_year,affiliation_type,affiliation_role,gmt_modified,department,visibility,city,region,start_date,end_date,is_phd,is_master
0,0000-0002-9418-3797,EG,Ain Shams University,68791,None,None,EDUCATION,PhD,1448730441029,Horticulture,PUBLIC,Cairo,None,None,None,True,False
1,0000-0002-9418-3797,EG,National Research Centre,68787,None,None,EMPLOYMENT,None,1448730520502,Plant Biotechnology,PUBLIC,Giza,None,None,None,False,False
2,0000-0001-6920-0797,BR,Universidade Federal de Pernambuco - UFPE,None,2017,2018,EDUCATION,Mestrando em Geografia - PPGEO,1520799749026,Ciências Geográficas - DCG/CFCH,PUBLIC,Recife,PE/NE,20170305,None,False,False
3,0000-0001-6920-0797,BR,Universidade Federal de Pernambuco - UFPE,None,2013,2017,EDUCATION,Licenciatura em Geografia,1520799751250,Ciências Geográficas - DCG/CFCH,PUBLIC,Recife,PE/NE,20130520,20170217,False,False
4,0000-0001-6920-0797,BR,Universidade Federal de Pernambuco - UFPE,133639,2017,2018,EMPLOYMENT,REPRESENTANTE DISCENTE DO MESTRADO TURMA 2017-...,1520800374502,Departamento de Ciências Geográficas - DCG,PUBLIC,Recife,PE/NE,20170527,None,False,True


In [180]:
len(df), df.orcid_id.nunique(), df.country.nunique()

(4432687, 1598770, 242)

We have about 4432687 affiliation observations from 1598770 ORCID profiles. Those affiliations are located in 242 countries, i.e. everywhere.

In [102]:
df.to_csv('/Users/qianshaoli/Documents/Academic/infm600/week5/teamproject/ORCID_migrations_2018_affiliation.csv', sep='\t', encoding = 'utf-8')
dfW.to_csv('/Users/qianshaoli/Documents/Academic/infm600/week5/teamproject/ORCID_migrations_2018_work.csv', sep='\t', encoding = 'utf-8')

Next, let's create a data set that summarizes important information about each person.

This dataframe will be indexed by orcid_id: one row per person. And let's start by generating a phd_year column with the year in which they completed their PhD.

Note: The value of phd_year will be 2018 for ongoing PhD students.

In [103]:
people = pd.DataFrame(index = df.orcid_id.unique())
people.index.name = 'orcid_id'
people["phd_year"] = df[(df.is_phd) 
                        & (df.affiliation_type == "EDUCATION")
                       ].groupby("orcid_id").end_year.max()
people.head()

,phd_year
orcid_id,
0000-0002-9418-3797,NaN
0000-0001-6920-0797,NaN
0000-0002-2917-0797,NaN
0000-0003-1280-4797,NaN
0000-0002-7543-3797,NaN


In [104]:
len(people)

1627845

There you have the year (if any) of 638568 PhD out of the 1598770 ORCID profiles that include affiliations. 

Now let's determine each person's country of residence in 2018.

In [105]:
today_countries = df[df.end_year == 2018].drop_duplicates(subset = 'orcid_id', keep = 'first')
today_countries = today_countries.set_index("orcid_id").country
today_countries.name = "country_2018"
today_countries.head()

orcid_id
0000-0001-6920-0797    BR
0000-0002-2917-0797    SE
0000-0003-1280-4797    ID
0000-0002-7543-3797    CN
0000-0002-0971-9797    US
Name: country_2018, dtype: object

In [106]:
people = pd.merge(people, today_countries.to_frame(), how = 'left',
                  left_index = True, right_index = True)
people.head()

,phd_year,country_2018
orcid_id,,
0000-0002-9418-3797,NaN,NaN
0000-0001-6920-0797,NaN,BR
0000-0002-2917-0797,NaN,SE
0000-0003-1280-4797,NaN,ID
0000-0002-7543-3797,NaN,CN


In [107]:
len(people), people.index.nunique()

(1627845, 1627845)

Good. So now we can see where each person ended up in 2018, if they have an affiliation that ended in 2018 or is ongoing.

We don't know the true nationalities of people with ORCID profiles, but we can infer them from the country of their earliest affiliation. So let's find the year and country of each person's earliest affiliation.

In [108]:
people['earliest_year'] = df.groupby("orcid_id").start_year.min()
people.head()

,phd_year,country_2018,earliest_year
orcid_id,,,
0000-0002-9418-3797,NaN,NaN,NaN
0000-0001-6920-0797,NaN,BR,2013
0000-0002-2917-0797,NaN,SE,1999
0000-0003-1280-4797,NaN,ID,2001
0000-0002-7543-3797,NaN,CN,1979


In [109]:
earliest_affs = df[df.start_year == df.groupby('orcid_id').start_year.transform(min)]
earliest_affs = earliest_affs.drop_duplicates(subset = 'orcid_id', keep = 'first').set_index('orcid_id').country
earliest_affs.name = 'earliest_country'
people = pd.merge(people, earliest_affs.to_frame(), how = 'left',right_index = True, left_index = True)
people.head()

,phd_year,country_2018,earliest_year,earliest_country
orcid_id,,,,
0000-0002-9418-3797,NaN,NaN,NaN,NaN
0000-0001-6920-0797,NaN,BR,2013,BR
0000-0002-2917-0797,NaN,SE,1999,RU
0000-0003-1280-4797,NaN,ID,2001,ID
0000-0002-7543-3797,NaN,CN,1979,CN


Good. Now let's mark people who obtained a PhD, and let's list the country in which they got their PhD.

In [110]:
people['has_phd'] = df.groupby('orcid_id').is_phd.max()
people.head()

,phd_year,country_2018,earliest_year,earliest_country,has_phd
orcid_id,,,,,
0000-0002-9418-3797,NaN,NaN,NaN,NaN,True
0000-0001-6920-0797,NaN,BR,2013,BR,False
0000-0002-2917-0797,NaN,SE,1999,RU,False
0000-0003-1280-4797,NaN,ID,2001,ID,False
0000-0002-7543-3797,NaN,CN,1979,CN,False


In [111]:
people['has_master'] = df.groupby('orcid_id').is_master.max()
people.head()

,phd_year,country_2018,earliest_year,earliest_country,has_phd,has_master
orcid_id,,,,,,
0000-0002-9418-3797,NaN,NaN,NaN,NaN,True,False
0000-0001-6920-0797,NaN,BR,2013,BR,False,True
0000-0002-2917-0797,NaN,SE,1999,RU,False,False
0000-0003-1280-4797,NaN,ID,2001,ID,False,True
0000-0002-7543-3797,NaN,CN,1979,CN,False,True


In [112]:
phd_country = df[(df.affiliation_type == 'EDUCATION') & (df.is_phd)]
phd_country = phd_country.drop_duplicates(subset = 'orcid_id', 
                                      keep = 'first').set_index('orcid_id').country
phd_country.name = 'phd_country'
people = pd.merge(people, phd_country.to_frame(), how = 'left',
                  right_index = True, left_index = True)

master_country = df[(df.affiliation_type == 'EDUCATION') & (df.is_master)]
master_country = master_country.drop_duplicates(subset = 'orcid_id', 
                                      keep = 'first').set_index('orcid_id').country
master_country.name = 'master_country'
people = pd.merge(people, master_country.to_frame(), how = 'left',
                  right_index = True, left_index = True)
people.head()

,phd_year,country_2018,earliest_year,earliest_country,has_phd,has_master,phd_country,master_country
orcid_id,,,,,,,,
0000-0002-9418-3797,NaN,NaN,NaN,NaN,True,False,EG,NaN
0000-0001-6920-0797,NaN,BR,2013,BR,False,True,NaN,NaN
0000-0002-2917-0797,NaN,SE,1999,RU,False,False,NaN,NaN
0000-0003-1280-4797,NaN,ID,2001,ID,False,True,NaN,ID
0000-0002-7543-3797,NaN,CN,1979,CN,False,True,NaN,CN


In [121]:
#add biography
dfPnew=dfP
peopleNew = pd.merge(people, dfPnew.set_index('orcid_id'), how = 'left',
                  right_index = True, left_index = True)
peopleNew.head()

,phd_year,country_2018,earliest_year,earliest_country,has_phd,has_master,phd_country,master_country,biography,keywords
orcid_id,,,,,,,,,,
0000-0002-9418-3797,NaN,NaN,NaN,NaN,True,False,EG,NaN,NaN,NaN
0000-0001-6920-0797,NaN,BR,2013,BR,False,True,NaN,NaN,NaN,NaN
0000-0002-2917-0797,NaN,SE,1999,RU,False,False,NaN,NaN,None,"|finance, computer science, derivatives"
0000-0003-1280-4797,NaN,ID,2001,ID,False,True,NaN,ID,NaN,NaN
0000-0002-7543-3797,NaN,CN,1979,CN,False,True,NaN,CN,Prof. D. A. Sun received his PhD in geotechnic...,"|Unsaturated soil mechanics, Constitutive mode..."


In [122]:
peopleNew.sample(30)

,phd_year,country_2018,earliest_year,earliest_country,has_phd,has_master,phd_country,master_country,biography,keywords
orcid_id,,,,,,,,,,
0000-0002-2887-520X,2004,IT,1991,IT,True,True,IT,IT,None,|Surface Science|Surface Analysis|XPS|AES|UPS
0000-0002-6227-6761,NaN,NaN,2008,CA,False,False,NaN,NaN,NaN,NaN
0000-0002-7603-8916,NaN,NL,2012,NL,False,False,NaN,NaN,NaN,NaN
0000-0002-4619-2277,NaN,US,inf,NaN,True,False,US,NaN,NaN,NaN
0000-0002-1038-9545,NaN,CN,1992,CN,False,False,NaN,NaN,NaN,NaN
0000-0003-2130-6669,2017,NaN,2007,IT,True,True,IT,IT,NaN,NaN
0000-0002-4127-1988,NaN,NaN,NaN,NaN,False,True,NaN,GB,NaN,NaN
0000-0002-2106-9643,2018,GB,1998,GB,True,True,GB,GB,None,"|politics, participation, decision making, ren..."
0000-0002-3871-8304,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN


Let's get some summary statistics based on these biographical data...

In [137]:
# total people who have both an earliest country and a 2018 country
len(peopleNew[(peopleNew.earliest_country.notnull()) & 
           (peopleNew.country_2018.notnull())])

1081169

1054586 people who have both an earliest country and a 2018 country

In [192]:
# total people in 2018 in a country different from their earliest country
len(people[(people.earliest_country.notnull()) & 
           (people.country_2018.notnull()) &
           (people.country_2018 != people.earliest_country)])

148416

So about 14.07% of people (148416 / 1054586) live in a country in 2018 different from their earliest affiliation.

Let's look at the distribution of PhD degrees.

In [193]:
len(people), len(people[people.has_phd])

(1598770, 705948)

44%(705948/1598770) of the ORCID profiles that include any affiliations list a PhD degree.

Where did they get those PhD degrees?

In [194]:
phds = people[people.has_phd].phd_country.value_counts()
phds.name = "PhDs"
phds = phds.to_frame()
phds["% of total"] = phds.PhDs / len(people[people.has_phd]) * 100
phds.head(10)

,PhDs,% of total
US,142763,20.222878
GB,60699,8.598225
IN,42599,6.034297
CN,40825,5.783004
ES,37947,5.375325
AU,26983,3.822236
BR,26062,3.691773
IT,19129,2.709690
FR,18772,2.659119
CA,16482,2.334733


PhDs  % of total
US  142763   20.222878
GB   60699    8.598225
IN   42599    6.034297
CN   40825    5.783004
ES   37947    5.375325
AU   26983    3.822236
BR   26062    3.691773
IT   19129    2.709690
FR   18772    2.659119
CA   16482    2.334733
PT   14583    2.065733
JP   13933    1.973658
DE   13710    1.942069
TR   11811    1.673069
MX   11746    1.663862
KR   11213    1.588361

So of the 705948 people with a PhD, 20% got it in the US, 8.5% in the UK, 6.0% in India, 5.7% in China, and 5.3% in Spain.

NOTE: Clearly some biases in the rate at which people from various countries have created ORCID profiles for themselves. Proceed with caution.

Save the people data to local directory. They are ready for play. Have fun!

In [138]:
peopleNew.to_csv('/Users/qianshaoli/Documents/Academic/infm600/week5/teamproject/ORCID_migrations_2018_person.csv',  sep='\t', encoding = 'utf-8')

In [139]:
len(peopleNew), len(people[peopleNew.has_phd]),len(people[peopleNew.has_master])

(1627845, 730177, 543582)